In [9]:
import numpy as np
import sympy as sp

import pyoti.sparse as oti

# Set pyoti to print all coefficients.
oti.set_printoptions(terms_print=-1)
np.set_printoptions(linewidth=120)

# tseuqlib
import tseuqlib as tuq
import tseuqlib.rv_moments as rv
import tseuqlib.oti_moments as moti
import tseuqlib.oti_util as uoti

# Affine arithmetic library
import affapy as af

In this example, we will explore the following function:

$$
y = x^{2} + \sin (x), \quad x \sim \mathcal{U}[0.99, 1.01]
$$

The objective is to compute the analytical expectations and moments and compare with TSEUQlib, interval analysis, and monte carlo sampling.


First, let's obtain the analytical expectation and moments using SymPy.

$$
\mathbb{E}[y] = \int_{0.99}^{1.01} g(x) f_{x}(x) dx =  \int_{0.99}^{1.01} (x^{2} + \sin (x)) (\frac{1}{1.01-0.99})(x) dx
$$

In [10]:
x = sp.Symbol('x')

# Define the function and PDF of uniform distribution on [0.99, 1.01] (99/100, 101/100)
a, b = sp.Rational(99, 100), sp.Rational(101, 100)
f = x**2 + sp.sin(x)
pdf = sp.Rational(1, 1) / (b - a)  # 1/(b-a) = 50

# E[y] = integral of f(x) * pdf over [a, b]
E_y = sp.integrate(f * pdf, (x, a, b))
E_y_simplified = sp.simplify(E_y)
E_y_float = float(E_y_simplified)

print("Exact E[y]:", E_y_simplified)
print("Numerical E[y]:", E_y_float)

Exact E[y]: -50*cos(101/100) + 30001/30000 + 50*cos(99/100)
Numerical E[y]: 1.8414902936949389


And similarly, we can compute $\text{Var}[y]$

$$
\text{Var}[y] = \mathbb{E}[y^2] -(\mathbb{E}[y])^{2}
$$

We just need to compute the first term now...

$$
\mathbb{E}[y^2] =  \int_{0.99}^{1.01} (x^{2} + \sin (x))^{2} (\frac{1}{1.01-0.99})(x) dx
$$


In [11]:
f2 = (x**2 + sp.sin(x))**2

# E[y^2] = integral of f(x) * pdf over [a, b]
E_y2 = sp.integrate(f2 * pdf, (x, a, b))
E_y2_simplified = sp.simplify(E_y2)
E_y2_float = float(E_y2_simplified)

print("Exact E[y^2]:", E_y2_simplified)
print("Numerical E[y^2]:", E_y2_float)

# Compute Var[y]
Var_y = E_y2_float - E_y_float**2
print("Var[y]:", Var_y)

Exact E[y^2]: -198*sin(99/100) - 10199*cos(99/100)/100 - 25*sin(101/50)/2 + 750100001/500000000 + 25*sin(99/50)/2 + 9799*cos(101/100)/100 + 202*sin(101/100)
Numerical E[y^2]: 3.391301605682763
Var[y]: 0.00021510391009060825


Now we will compute the same but using a 1st order TSE

In [12]:
def func(x, alg=oti):
    return x[0]**2 + alg.sin(x[0])

# Use a first order TSE to estimate the expectation and variance
tse_order = 1

# Define x as a uniform distribution
rv_pdf_name = np.array(['U'])

# Define hyperparameters for uniform dist.
rv_a = np.array([0.99]) 
rv_b = np.array([1.01])

# Compute expectation of the random variable
rv_params = rv.rv_mean_from_ab([rv_pdf_name, rv_a, rv_b])

# Random variable(s) with nominal values at the mean and perturbed with OTI imaginary directions
n_dim = 1
x = [0]*n_dim
for d in range(n_dim):
    x[d] = rv_params[3][d] + oti.e(int(d+1), order=tse_order)
    
# Evaluate the function at the mean
f0 = func(x)

# generate moments of the random variables
rv_moments_order = int(tse_order*2) #TODO: generalize for moments past 12th order

# rv moments object
rv_moments = rv.rv_central_moments(rv_params, rv_moments_order)
rv_moments.compute_central_moments()

# Build the joint distribution from uncorrelated variables.
rv_mu_joint = uoti.build_rv_joint_moments(rv_moments.rv_mu)

# Create oti_moments object
oti_mu = moti.tse_uq(n_dim, rv_mu_joint)

# Expectation
mu_y   = oti_mu.expectation(f0)
print('mu_y =', mu_y)

# Variance
mu2     = oti_mu.central_moment(f0, 2)
print('mu2 =', mu2)

mu_y = 1.8414709848078965
mu2 = 0.00021510452683996668


In [13]:
# Relative error in [%]
rel_error_E_y = np.abs((E_y_float - mu_y)/(E_y_float)) * 100
print(f'rel_error_E_y: {rel_error_E_y} [%]')

rel_error_Var_y = np.abs((Var_y - mu2)/(mu2)) * 100
print(f'rel_error_Var_y: {rel_error_Var_y} [%]')

rel_error_E_y: 0.001048546772605382 [%]
rel_error_Var_y: 0.00028672077128546986 [%]


Now, let's simply increase the TSE order to 2 to utilize higher-order terms to approximate expectation and variance

In [14]:
# Use a 2nd order TSE to estimate the expectation and variance
tse_order = 2

# Define x as a uniform distribution
rv_pdf_name = np.array(['U'])

# Define hyperparameters for uniform dist.
rv_a = np.array([0.99]) 
rv_b = np.array([1.01])

# Compute expectation of the random variable
rv_params = rv.rv_mean_from_ab([rv_pdf_name, rv_a, rv_b])

# Random variable(s) with nominal values at the mean and perturbed with OTI imaginary directions
n_dim = 1
x = [0]*n_dim
for d in range(n_dim):
    x[d] = rv_params[3][d] + oti.e(int(d+1), order=tse_order)
    
# Evaluate the function at the mean
f0 = func(x)

# generate moments of the random variables
rv_moments_order = int(tse_order*2) #TODO: generalize for moments past 12th order

# rv moments object
rv_moments = rv.rv_central_moments(rv_params, rv_moments_order)
rv_moments.compute_central_moments()

# Build the joint distribution from uncorrelated variables.
rv_mu_joint = uoti.build_rv_joint_moments(rv_moments.rv_mu)

# Create oti_moments object
oti_mu = moti.tse_uq(n_dim, rv_mu_joint)

# Expectation
mu_y   = oti_mu.expectation(f0)
print('mu_y =', mu_y)

# Variance
mu2     = oti_mu.central_moment(f0, 2)
print('mu2 =', mu2)

mu_y = 1.8414902936248163
mu2 = 0.00021510482510429533


In [15]:
# Relative error in [%]
rel_error_E_y = np.abs((E_y_float - mu_y)/(E_y_float)) * 100
print(f'rel_error_E_y: {rel_error_E_y} [%]')

rel_error_Var_y = np.abs((Var_y - mu2)/(Var_y)) * 100
print(f'rel_error_Var_y: {rel_error_Var_y} [%]')

rel_error_E_y: 3.807925279532377e-09 [%]
rel_error_Var_y: 0.00042538217305963295 [%]


Now let's use ``affapy`` for this problem. We will explore using ``Interval`` and ``Affine`` arithmetic. Let's stick to just 1st-order TSE.

Let's define $\mu \in [0.99, 1.01]$. The expected value using a 1st-order TSE is simply

$$
\mathbb{E}[y] \approx f(\mu)
$$

In [25]:
from affapy.ia import Interval
from affapy.aa import Affine

# Define the function only with numpy operations
def func_interval(x):
    return x**2 + np.sin(x)


# Defining mu as an affine interval
mu_interval = Interval(0.99, 1.01)
mu_affine = Affine([0.99, 1.01])

# Evaluating using interval and affine arithmetic
E_y_interval = func_interval(mu_interval)
E_y_affine = func_interval(mu_affine)

# Print results
print(f"E[y] (IA): {E_y_interval}")
print(f"E[y] (AA): {E_y_affine.interval}")

E[y] (IA): [1.81612597860052, 1.86693184461802]
E[y] (AA): [1.81592593449448, 1.86697997230603]
